<a href="https://colab.research.google.com/github/arashkhgit/Data-Engineering/blob/main/STDP_train_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [299]:
from skimage.io import imread_collection
from skimage.io import imshow_collection
import cv2
from google.colab.patches import cv2_imshow
import struct

from PIL import Image as im
import numpy as np

i=6
#your path 
col_dir = '/content/train/9/*.png'

#creating a collection with the available images
col = imread_collection(col_dir)

In [300]:
!pip install unrar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [301]:
#!unrar x /content/mnist-pngs-dataset.rar

In [302]:
len(col)

5949

In [303]:
col[0].shape

(28, 28)

In [304]:
#for i in range(len(col)):
#  cv2_imshow(col[i])

In [305]:
flatten=[]

In [306]:
for i in range(len(col)): 
  flatten.append(col[i].flatten())

In [307]:
len(flatten[0])

784

In [308]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [309]:
def DeNormalizeData(X,data):
    return (X*(np.max(data)-np.min(data))+np.min(data))

In [310]:
for i in range(len(col)): 
  flatten[i]=NormalizeData(flatten[i])

In [311]:
sum=0

In [312]:
for i in range(len(col)): 
  sum=sum+flatten[i]
sum=sum/len(col)

In [313]:
sum

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [314]:
# Function for converting decimal to binary
def float_bin(my_number, places = 3):
    my_whole, my_dec = str(my_number).split(".")
    my_whole = int(my_whole)
    res = (str(bin(my_whole))+".").replace('0b','')
 
    for x in range(places):
        my_dec = str('0.')+str(my_dec)
        temp = '%1.20f' %(float(my_dec)*2)
        my_whole, my_dec = temp.split(".")
        res += my_whole
    return res
 
 
 
def IEEE754(n) :
    # identifying whether the number
    # is positive or negative
    sign = 0
    if n < 0 :
        sign = 1
        n = n * (-1)
    p = 30
    # convert float to binary
    dec = float_bin (n, places = p)
 
    dotPlace = dec.find('.')
    onePlace = dec.find('1')
    # finding the mantissa
    if onePlace > dotPlace:
        dec = dec.replace(".","")
        onePlace -= 1
        dotPlace -= 1
    elif onePlace < dotPlace:
        dec = dec.replace(".","")
        dotPlace -= 1
    mantissa = dec[onePlace+1:]
 
    # calculating the exponent(E)
    exponent = dotPlace - onePlace
    exponent_bits = exponent + 127
 
    # converting the exponent from
    # decimal to binary
    exponent_bits = bin(exponent_bits).replace("0b",'')
 
    mantissa = mantissa[0:23]
 
    # the IEEE754 notation in binary    
    final = str(sign) + exponent_bits.zfill(8) + mantissa
 
    # convert the binary to hexadecimal
    hstr = '0x%0*X' %((len(final) + 3) // 4, int(final, 2))
    return (hstr)

In [315]:
def float_to_bin(num):
    # Convert a float number to its binary representation
    return bin(struct.unpack('!I', struct.pack('!f', num))[0])[2:].zfill(32)
    
def float_to_hex(num):
    # Convert a float number to its hexadecimal representation
    return "0x" + str(hex(struct.unpack('!I', struct.pack('!f', num))[0])[2:].zfill(8))

def ieee_754_hex(num):
    # Convert a float number to its IEEE 754 hexadecimal representation
    binary = float_to_bin(num)
    sign_bit = binary[0]
    exponent_bits = binary[1:9]
    fraction_bits = binary[9:]
    exponent = int(exponent_bits, 2) - 127
    mantissa = 1.0
    for i in range(len(fraction_bits)):
        mantissa += int(fraction_bits[i]) * 2.0 ** (-(i+1))
    if sign_bit == '1':
        mantissa = -mantissa
    hex_mantissa = hex(int(abs(mantissa) * 2 ** 23))[2:].zfill(6)
    if mantissa < 0:
        hex_mantissa = hex((int(hex_mantissa, 16) ^ int('ffffff', 16)) + 1)[2:].zfill(6)
    hex_exponent = hex(exponent + 127)[2:].zfill(2)
    hex_sign = sign_bit
    return "0x" + str((hex_sign + hex_exponent + hex_mantissa).upper())

In [316]:
empty_array = []

In [317]:
for i in range(len(sum)):
  if sum[i]==0 :
    empty_array.append('0x00000000')
  else :
    #print(flatten[i])
    empty_array.append(float_to_hex(float(sum[i])))

In [318]:
print((empty_array))

['0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00000000', '0x00

In [319]:
np.savetxt('test.txt', flatten,fmt='%1.3f',newline=' ') 

In [320]:
file = open('items.txt','w')
for item in empty_array:
	file.write(item+"\n")
file.close()

In [321]:
unflatt=np.reshape(sum, (28,28))

In [322]:
np.shape(unflatt)

(28, 28)

In [323]:
img = np.zeros([28,28,3])

img[:,:,0] = col[0]*64/255.0
img[:,:,1] = col[0]*128/255.0
img[:,:,2] = col[0]*192/255.0

#cv2.imwrite('color_img.jpg', img)
#cv2_imshow(img)

In [324]:
norm_image = cv2.normalize(unflatt, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
norm_image = norm_image.astype(np.uint8)

In [325]:
norm_image

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   4,   7,
         11,  13,  13,  11,   9,   6,   3,   2,   0,   0,   0,   0,   0,
          0,   0],
       [  

In [326]:
data = im.fromarray(np.reshape(norm_image, (28,28)))
data=data.convert('RGB')
data.save('gfg_dummy_pic.png')

In [327]:
sum[526]

0.009857145315231963

In [328]:
empty_array[710]

'0x3e15f76a'

In [329]:
suppressedNum=f"{2.10557481743e-38:.8f}"
print(suppressedNum)

0.00000000


In [330]:
len(sum)

784